사칙연산게임
- 좌우로 나누어서 좌측에는 연산대상 숫자를 랜덤으로 일정시간 보여주고
- 우측상단에 해당 연산을 해서 기록하면
- 우측하단에는 정답과 오답으로 나누어서 보여준다.
- 다시하겠습니까? 그만하시겠습니까? 
- 스코어와 정답률을 제공한다.
- 어느 부분에 어떤 겜을 넣으면 좋을 것도 같다


In [19]:


import sys
import math
import pygame
import time
import random
from pygame.locals import QUIT,KEYDOWN,K_LEFT,K_RIGHT,Rect

pygame.init()
pygame.key.set_repeat(1,1)
SURFACE = pygame.display.set_mode((800,600))
FPSCLOCK = pygame.time.Clock()
# pygame.mixer.music.load("Escape.mp3")
# pygame.mixer.music.play()
# 종료 pygame.mixer.music.stop()
# hit_sound = pygame.mixer.Sound("bang.wav")   # 짧은 wav/mp3 파일
eat_sound = pygame.mixer.Sound("eat.wav")   # 짧은 wav/mp3 파일
# 실행 hit_sound.play()
result_num=0  #결과값


def left_question():
    left_list=[]
    first_num = str(random.randint(1,10))
    symbol=  str(random.choice(['+', '-', '*']))
    second_num= str(random.randint(1,10))
    left_list=[first_num,symbol,second_num]
    return left_list

def main():
    myfont= pygame.font.SysFont(None,36)
    middlefont=pygame.font.SysFont(None,72)
    largefont= pygame.font.SysFont(None,250)


    mess_clear = myfont.render("Cleared !!",True,(255,255,0))
    mess_over = myfont.render("Game Over !!",True,(255,255,0))
    tot_count=0
    ok_count=0
    ok_rate=0    #정답률산정

    while True:
        start_time = pygame.time.get_ticks()  # 시작 시각 (밀리초 단위)
        left_list=left_question()
        if left_list[1] =='+':   #결과값을 구하는 로직
            result_num = int(left_list[0]) + int(left_list[2])
        elif left_list[1] == '-':
            result_num = int(left_list[0]) - int(left_list[2])
        elif left_list[1] == '*':
            result_num = int(left_list[0]) * int(left_list[2])

        eat_sound.play()

        for i in range(3):
            while True:
                for event in pygame.event.get():
                    if event.type == QUIT:
                        pygame.quit()
                        sys.exit()
                    elif event.type == KEYDOWN:
                        pass
                
                SURFACE.fill((0,0,0))
                q_left=largefont.render(left_list[i],True,(255,255,0))
                SURFACE.blit(q_left,(350,200))
                try:
                    ok_rate=round((ok_count/tot_count)*100,1)
                    okraterect=myfont.render(f"next : space_ bar   correct:{ok_rate} %  {tot_count}",True,(255,255,0))
                    pygame.draw.rect(SURFACE, (0, 0, 0), (300, 30, 500, 40))  # (x, y, w, h)
                    SURFACE.blit(okraterect,(300,30))      
                except:
                    pass   
                pygame.display.flip()

                FPSCLOCK.tick(30)
                # 5초(5000ms) 지나면 while문 탈출
                if pygame.time.get_ticks() - start_time > 1000:
                    start_time = pygame.time.get_ticks()  # 시작 시각 (밀리초 단위)
                    eat_sound.play()

                    break
            

        # 입력 박스: (500, 200) 위치에 고정
        SURFACE.fill((0,0,0))

        input_box = pygame.Rect(500, 200, 160, 80)
        bg_color = (20, 20, 20)
        box_bg = (35, 35, 35)
        border_inactive = pygame.Color('lightskyblue3')
        border_active = pygame.Color('dodgerblue2')
        active = True
        text = ""
        font_size=30
        max_len = 10   # 길이 제한(선택)

        # 중요: 키 자동반복 끄기(혹시 어딘가에서 켰다면)
        pygame.key.set_repeat()  # 인자 없이 호출하면 반복 비활성

        def append_sanitized(existing: str, incoming: str, limit: int) -> str:
            """맨 앞의 '-' 1개만 허용, 나머지는 숫자만. 붙여넣기도 처리.""" 
            out = existing
            for ch in incoming:
                if ch == '-':
                    if len(out) == 0:            # 맨 앞에만 허용
                        out = '-' + out
                elif ch.isdigit():
                    if len(out) < limit:
                        out += ch
                # 그 외 문자는 무시
            return out[:limit]
            
        running = True
        continue_a = True
        # start_time = pygame.time.get_ticks()  # 시작 시각 (밀리초 단위)

        while running:
            # active = True
            for ev in pygame.event.get():
                if ev.type == pygame.QUIT:
                    running = False

                elif ev.type == pygame.TEXTINPUT and active:
                    text = append_sanitized(text, ev.text, max_len)

                elif ev.type == pygame.KEYDOWN and active:
                    if ev.key == pygame.K_RETURN:
                        # 빈 문자열이나 단독 '-'는 무시
                        if text not in ("", "-"):
                            tot_count +=1
                            eat_sound.play()

                            try:
                                if text== str(result_num):
                                    correrct=middlefont.render("Ok!",True,(255,0,0))
                                    SURFACE.blit(correrct,(500,400))
                                    ok_count+=1
                                else:
                                    correrct=middlefont.render("miss!",True,(0,255,0))
                                    
                                    SURFACE.blit(correrct,(500,400))

                            except ValueError:
                                pass
                        text = ""
                    elif ev.key == pygame.K_BACKSPACE:
                        text = text[:-1]
                    elif ev.key == pygame.K_MINUS:  # 일부 키보드에서 보조용
                        text = append_sanitized(text, "-", max_len)
                    elif ev.key == pygame.K_ESCAPE:
                        active = False
                    elif ev.key == pygame.K_SPACE:
                        continue_a=False

            if continue_a == False:
                break
            # --- 그리기 ---
            # (1) 전체 화면을 매 프레임 지우지 않는 게임이라면,
            #     최소한 입력박스 영역만은 배경색으로 먼저 덮어주세요.
            pygame.draw.rect(SURFACE, bg_color, input_box)     # 박스 영역 클리어

            # (2) 박스 배경 + 테두리
            pygame.draw.rect(SURFACE, box_bg, input_box)
            pygame.draw.rect(SURFACE, border_active if active else border_inactive, input_box, 2)

            # (3) 텍스트
            surf = middlefont.render(text, True, (220, 220, 220))
            SURFACE.blit(surf, (input_box.x + 10, input_box.y + 10))

            # (선택) 커서 깜빡임
            if active and (pygame.time.get_ticks() // 500) % 2 == 0:
                caret_x = input_box.x + 10 + surf.get_width() + 2
                caret_y = input_box.y + 10
                pygame.draw.line(SURFACE, (220, 220, 220), (caret_x, caret_y), (caret_x, caret_y + surf.get_height()), 2)
            try:
                ok_rate=round((ok_count/tot_count)*100,1)
                okraterect=myfont.render(f"next : space_ bar   correct:{ok_rate} %  {tot_count}",True,(255,255,0))
                pygame.draw.rect(SURFACE, (0, 0, 0), (300, 30, 500, 40))  # (x, y, w, h)
                SURFACE.blit(okraterect,(300,30))      
            except:
                pass  
            pygame.display.flip()
            FPSCLOCK.tick(30)
    # pygame.quit()
    # sys.exit()            
if __name__ =='__main__':
    main()

SystemExit: 

In [8]:
a=round((1/3)*100,1)
a

33.3

In [1]:
tot_count

NameError: name 'tot_count' is not defined

In [1]:
import pygame
pygame.init()

W, H = 800, 600
screen = pygame.display.set_mode((W, H))
clock = pygame.time.Clock()
font = pygame.font.SysFont(None, 48)

# 입력 박스: (500, 200) 위치에 고정
input_box = pygame.Rect(500, 200, 160, 56)
bg_color = (20, 20, 20)
box_bg = (35, 35, 35)
border_inactive = pygame.Color('lightskyblue3')
border_active = pygame.Color('dodgerblue2')
active = False
text = ""
max_len = 10   # 길이 제한(선택)

# 중요: 키 자동반복 끄기(혹시 어딘가에서 켰다면)
pygame.key.set_repeat()  # 인자 없이 호출하면 반복 비활성

running = True
continue_a= True
while running:
    for ev in pygame.event.get():
        if ev.type == pygame.QUIT:
            running = False

        elif ev.type == pygame.MOUSEBUTTONDOWN:
            active = input_box.collidepoint(ev.pos)

        # 숫자 입력은 TEXTINPUT에서만 처리 → 키 반복/중복 입력 방지
        elif ev.type == pygame.TEXTINPUT and active:
            if ev.text.isdigit():
                if len(text) < max_len:
                    text += ev.text

        # 특수키는 KEYDOWN에서 처리(엔터/백스페이스/ESC)
        elif ev.type == pygame.KEYDOWN and active:
            if ev.key == pygame.K_RETURN:
                print("입력값:", text)   # 여기에서 게임 로직으로 넘기면 됨
                text = ""
            elif ev.key == pygame.K_BACKSPACE:
                text = text[:-1]
            elif ev.key == pygame.K_ESCAPE:
                active = False

    # --- 그리기 ---
    # (1) 전체 화면을 매 프레임 지우지 않는 게임이라면,
    #     최소한 입력박스 영역만은 배경색으로 먼저 덮어주세요.
    pygame.draw.rect(screen, bg_color, input_box)     # 박스 영역 클리어

    # (2) 박스 배경 + 테두리
    pygame.draw.rect(screen, box_bg, input_box)
    pygame.draw.rect(screen, border_active if active else border_inactive, input_box, 2)

    # (3) 텍스트
    surf = font.render(text, True, (220, 220, 220))
    screen.blit(surf, (input_box.x + 10, input_box.y + 10))

    # (선택) 커서 깜빡임
    if active and (pygame.time.get_ticks() // 500) % 2 == 0:
        caret_x = input_box.x + 10 + surf.get_width() + 2
        caret_y = input_box.y + 10
        pygame.draw.line(screen, (220, 220, 220), (caret_x, caret_y), (caret_x, caret_y + surf.get_height()), 2)

    pygame.display.flip()
    clock.tick(60)

pygame.quit()


c:\Users\고창섭\AppData\Local\Programs\Python\Python313\Lib\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


pygame 2.6.1 (SDL 2.28.4, Python 3.13.7)
Hello from the pygame community. https://www.pygame.org/contribute.html
입력값: 20


In [10]:
import pygame

pygame.init()
W, H = 600, 300
screen = pygame.display.set_mode((W, H))
pygame.display.set_caption("Rotate 3 chars at one spot")
clock = pygame.time.Clock()

# 글자 준비
font = pygame.font.SysFont(None, 160)
letters = ["A", "B", "C"]
idx = 0

# 표시 위치와 배경색
pos = (W//2, H//2)  # 화면 중앙 기준 (정확한 중앙 배치는 rect로 조정)
bg = (20, 20, 30)
screen.fill(bg)

# 0.5초마다 바꾸기 위한 타이머
CHANGE = pygame.USEREVENT + 1
pygame.time.set_timer(CHANGE, 500)

# 직전 글자 영역 기억해서 부분 지우기
prev_rect = None

running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

        elif event.type == CHANGE:
            # 이전 글자 영역만 깔끔히 지우기
            if prev_rect:
                screen.fill(bg, prev_rect)

            # 새 글자 렌더링
            surf = font.render(letters[idx], True, (255, 255, 255))
            rect = surf.get_rect(center=pos)

            # blit 하고 화면 업데이트
            screen.blit(surf, rect)
            pygame.display.update(prev_rect if prev_rect else rect)  # 처음엔 prev_rect가 없으니 rect로
            pygame.display.update(rect)

            # 다음 인덱스 준비
            prev_rect = rect
            idx = (idx + 1) % len(letters)

    clock.tick(60)

pygame.quit()


In [4]:
left_list=left_question()

In [6]:
type(left_list[0])

str

In [10]:
left_list=left_question()
for i in left_list:
    q_left=i
    print(i)
    type(q_left)
    time.sleep(5)


9
-
8


In [7]:
import random

a = random.randint(1, 10)
b = random.randint(1, 10)
op = random.choice(['+', '-', '*', '/'])

expression = f"{a} {op} {b}"
result = eval(expression)

print(f"문제: {expression} = ?")
print(f"정답: {result}")


문제: 6 / 7 = ?
정답: 0.8571428571428571
